# Telegram Meteora Pool Analyzer Report Generator

## 📊 Overview
This notebook generates automated liquidity pool analysis reports using Gateway's Meteora CLMM endpoints and distributes them via Telegram. It fetches real-time pool data, analyzes liquidity distribution, price movements, and generates comprehensive reports with charts.

## 🎯 Objectives
1. **Pool Data Fetching**: Connect to Gateway API and fetch Meteora pool information
2. **Liquidity Analysis**: Analyze bin distribution, liquidity concentration, and price ranges
3. **Price Monitoring**: Track current price, active bins, and price movements
4. **Report Generation**: Create formatted pool analysis reports with key metrics
5. **Telegram Integration**: Send reports via Telegram bot using notification system
6. **Chart Creation**: Generate liquidity depth charts and price visualizations

## 📋 Prerequisites
- Gateway service running locally (default: http://localhost:15888)
- Telegram bot configuration (bot token and chat ID in .env)
- QuantsLab notification system setup
- Environment variables configured (.env file)
- Plotly and pandas for analysis

## ⚠️ Important Notes
- **Data Source**: Uses Gateway API for real-time Meteora pool data
- **Default Pool**: SOL-USDC pool (configurable)
- **Update Interval**: Configurable interval for periodic updates
- **Rate Limiting**: Be mindful of API rate limits for both Gateway and Telegram

## 📈 Expected Outputs
- Real-time pool metrics and liquidity analysis
- Liquidity distribution charts showing bin concentrations
- Price range analysis with support/resistance levels
- CSV reports with detailed bin data
- Automated Telegram notifications with pool updates

In [178]:
# 🔌 Initialize Core Services and Libraries

## ⚙️ Configuration and Gateway API Client

import warnings
warnings.filterwarnings("ignore")

import sys
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
from datetime import datetime, timezone, timedelta
from dotenv import load_dotenv
from typing import Dict, List, Optional, Any
import json

# Import Gateway HTTP Client from hummingbot library
from hummingbot.core.gateway.gateway_http_client import GatewayHttpClient
from hummingbot.client.config.client_config_map import GatewayConfigMap

# Import QuantsLab notification system
from core.notifiers import NotificationManager, NotificationMessage

# Load environment variables
load_dotenv()

print("✅ Libraries imported successfully")
print("📊 Services available: Gateway HTTP Client (from hummingbot), Notification system")
print("⚙️ Environment variables loaded from .env")

✅ Libraries imported successfully
📊 Services available: Gateway HTTP Client (from hummingbot), Notification system
⚙️ Environment variables loaded from .env


In [187]:
# Configuration Parameters and Gateway Setup
GATEWAY_URL = os.getenv('GATEWAY_URL', 'http://localhost:15888')
NETWORK = 'mainnet-beta'
UPDATE_INTERVAL_SECONDS = 300  # 5 minutes between updates

# Pool addresses to analyze - add your pool addresses here
POOL_ADDRESSES = [
    '2sf5NYcY4zUPXUSmG6f66mskb24t5F8S11pC1Nz5nQT3',  # SOL-USDC
    'C8Gr6AUuq9hEdSYJzoEpNcdjpojPZwqG5MtQbeouNNwg',  # JUP-SOL
    '9d9mb8kooFfaD3SctgZtkxQypkshx6ezhbKio89ixyy2',  # TRUMP-USDC
    # Add more pool addresses here as needed
    # 'your_pool_address_here',
]

# Initialize Gateway Client
gateway_config = GatewayConfigMap()
gateway_config.gateway_api_host = GATEWAY_URL.replace('http://', '').replace('https://', '').split(':')[0]
gateway_config.gateway_api_port = str(GATEWAY_URL.split(':')[-1])
gateway_config.gateway_use_ssl = False

# Get the Gateway client instance
gateway_client = GatewayHttpClient.get_instance(gateway_config)

# Initialize notification manager
notification_manager = NotificationManager()
enabled_notifiers = notification_manager.get_enabled_notifiers()

# Initialize collections for Telegram sending
telegram_reports = {
    'text_reports': [],
    'chart_files': [],
    'csv_files': [],
    'report_message': None
}

# Global token cache to avoid re-fetching
token_cache = {}

# 📊 Meteora Pool Analyzer Class
class MeteorPoolAnalyzer:
    """Helper class for fetching and analyzing Meteora pools using Gateway HTTP Client"""
    
    def __init__(self, gateway_client: GatewayHttpClient):
        self.client = gateway_client
        
    async def fetch_pool_info(self, pool_address: str, network: str = 'mainnet-beta') -> Optional[Dict]:
        """Fetch Meteora pool information from Gateway"""
        try:
            # Make request to Gateway's Meteora endpoint
            response = await self.client.api_request(
                method="get",
                path_url="connectors/meteora/clmm/pool-info",
                params={
                    'network': network,
                    'poolAddress': pool_address
                }
            )
            return response
        except Exception as e:
            print(f"❌ Error fetching pool info: {e}")
            return None
    
    async def fetch_token_info(self, token_address: str, network: str = 'mainnet-beta') -> Optional[Dict]:
        """Fetch token information from Gateway with caching"""
        # Check cache first
        if token_address in token_cache:
            return token_cache[token_address]
            
        try:
            response = await self.client.api_request(
                method="get",
                path_url=f"tokens/{token_address}",
                params={
                    'chain': 'solana',
                    'network': network
                }
            )
            token_info = response.get('token', {})
            # Cache the result
            token_cache[token_address] = token_info
            return token_info
        except Exception as e:
            print(f"❌ Error fetching token info for {token_address}: {e}")
            # Cache the failure so we don't retry
            token_cache[token_address] = None
            return None
    
    def calculate_liquidity_distribution(self, bins_df, current_price: float, total_liquidity: float) -> Dict[str, float]:
        """Calculate liquidity distribution at different price ranges"""
        distributions = {}
        
        for pct in [5, 10, 15, 20]:
            lower_price = current_price * (1 - pct/100)
            upper_price = current_price * (1 + pct/100)
            
            bins_in_range = bins_df[
                (bins_df['price'] >= lower_price) & 
                (bins_df['price'] <= upper_price)
            ]
            
            liquidity_in_range = bins_in_range['total_value'].sum() if not bins_in_range.empty else 0
            pct_of_total = (liquidity_in_range / total_liquidity * 100) if total_liquidity > 0 else 0
            
            distributions[pct] = pct_of_total
            
        return distributions
    
    def analyze_pool(self, pool_data: Dict, metadata: Dict = None) -> Dict:
        """Analyze pool data and calculate metrics"""
        # Add metadata if provided
        if metadata:
            pool_data.update(metadata)
            
        # Calculate total liquidity
        price = pool_data['price']
        base_amount = pool_data['baseTokenAmount']
        quote_amount = pool_data['quoteTokenAmount']
        total_liquidity_usd = quote_amount + (base_amount * price)
        
        analysis = {
            'address': pool_data['address'],
            'pool_name': pool_data.get('pool_name', 'Unknown'),
            'base_token': pool_data.get('base_token_symbol', pool_data.get('base_token', 'BASE')),
            'quote_token': pool_data.get('quote_token_symbol', pool_data.get('quote_token', 'QUOTE')),
            'base_token_address': pool_data.get('baseTokenAddress'),
            'quote_token_address': pool_data.get('quoteTokenAddress'),
            'current_price': price,
            'fee_pct': pool_data['feePct'],
            'dynamic_fee_pct': pool_data.get('dynamicFeePct', pool_data['feePct']),
            'bin_step': pool_data.get('binStep', 0),
            'active_bin_id': pool_data.get('activeBinId'),
            'base_liquidity': base_amount,
            'quote_liquidity': quote_amount,
            'total_liquidity_usd': total_liquidity_usd,
            'num_bins': len(pool_data.get('bins', [])),
            'liquidity_concentration': 0,
            'liquidity_distribution': {},
            'price_range': {'min': 0, 'max': 0},
            'timestamp': datetime.now(timezone.utc)
        }
        
        # Analyze bins if available
        bins = pool_data.get('bins', [])
        if bins:
            bins_df = pd.DataFrame(bins)
            
            # Calculate price range
            analysis['price_range']['min'] = bins_df['price'].min()
            analysis['price_range']['max'] = bins_df['price'].max()
            
            # Calculate bin values
            bins_df['total_value'] = (
                bins_df['quoteTokenAmount'] + 
                bins_df['baseTokenAmount'] * price
            )
            
            total_liq = bins_df['total_value'].sum()
            
            # Calculate concentration around active bin
            if analysis['active_bin_id'] is not None:
                active_bin_id = analysis['active_bin_id']
                nearby_bins = bins_df[
                    (bins_df['binId'] >= active_bin_id - 5) &
                    (bins_df['binId'] <= active_bin_id + 5)
                ]
                
                nearby_liq = nearby_bins['total_value'].sum() if not nearby_bins.empty else 0
                
                if total_liq > 0:
                    analysis['liquidity_concentration'] = (nearby_liq / total_liq) * 100
            
            # Calculate liquidity distribution at different price ranges
            analysis['liquidity_distribution'] = self.calculate_liquidity_distribution(
                bins_df, price, total_liq
            )
            
            # Find key liquidity levels
            top_bins = bins_df.nlargest(5, 'total_value')
            analysis['key_levels'] = [
                {'price': row['price'], 'total_value': row['total_value']}
                for _, row in top_bins.iterrows()
            ]
            
            analysis['bins_df'] = bins_df
            
        return analysis

# Initialize the analyzer
pool_analyzer = MeteorPoolAnalyzer(gateway_client)

def format_number(value: float, decimals: int = 2, is_currency: bool = False) -> str:
    """Format numbers for display"""
    if is_currency:
        if value >= 1_000_000:
            return f"${value/1_000_000:.{decimals}f}M"
        elif value >= 1_000:
            return f"${value/1_000:.{decimals}f}K"
        else:
            return f"${value:.{decimals}f}"
    else:
        if value >= 1_000_000:
            return f"{value/1_000_000:.{decimals}f}M"
        elif value >= 1_000:
            return f"{value/1_000:.{decimals}f}K"
        else:
            return f"{value:.{decimals}f}"

def format_price(price: float) -> str:
    """Format price with appropriate decimal places based on magnitude"""
    if price >= 100:
        return f"${price:.2f}"
    elif price >= 10:
        return f"${price:.3f}"
    elif price >= 1:
        return f"${price:.4f}"
    elif price >= 0.1:
        return f"${price:.5f}"
    elif price >= 0.01:
        return f"${price:.6f}"
    elif price >= 0.001:
        return f"${price:.7f}"
    elif price >= 0.0001:
        return f"${price:.8f}"
    elif price > 0:
        # For very small prices, use scientific notation if needed
        if price < 0.00000001:
            return f"${price:.2e}"
        else:
            return f"${price:.10f}"
    else:
        return "$0.00"

async def generate_pool_report(analyses: List[Dict]) -> str:
    """Generate formatted pool analysis report for Telegram using cached token symbols"""
    
    if not analyses:
        return "⚠️ No pool data available for analysis"
    
    # Generate report timestamp
    report_time = datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M UTC')
    
    # Build the report message
    report_message = f"""🏊 <b>Meteora Pool Analysis Report</b>
📅 {report_time}
🔗 Network: Solana {NETWORK}
⚡ Powered by Gateway (via Hummingbot)

"""
    
    # Add analysis for each pool
    for analysis in analyses:
        # Use cached token symbols from initial pool fetching
        base_symbol = analysis['base_token']
        quote_symbol = analysis['quote_token']
        
        # Only try to fetch if we don't already have symbols and they're not addresses
        if analysis.get('base_token_address') and len(base_symbol) > 10:  # Likely an address, not symbol
            if analysis['base_token_address'] in token_cache:
                cached_info = token_cache[analysis['base_token_address']]
                if cached_info and 'symbol' in cached_info:
                    base_symbol = cached_info['symbol']
                    
        if analysis.get('quote_token_address') and len(quote_symbol) > 10:  # Likely an address, not symbol
            if analysis['quote_token_address'] in token_cache:
                cached_info = token_cache[analysis['quote_token_address']]
                if cached_info and 'symbol' in cached_info:
                    quote_symbol = cached_info['symbol']
        
        # Create token pair name with fee tier and bin step
        token_pair = f"{base_symbol}/{quote_symbol}"
        fee_tier = f"{analysis['fee_pct']:.2f}%"
        bin_step = analysis['bin_step']
        
        # Pool header with token pair, fee tier, and bin step
        report_message += f"""📊 <b>{token_pair} • {fee_tier} • Bin:{bin_step}</b>
<i>Pool: {analysis['pool_name']}</i>
━━━━━━━━━━━━━━━━━━━━
"""
        
        # Price and liquidity info with smart formatting
        price = analysis['current_price']
        price_str = format_price(price)
        report_message += f"""💰 <b>Price & Liquidity</b>
• Current Price: <code>{price_str}</code>
• Total Liquidity: <code>{format_number(analysis['total_liquidity_usd'], is_currency=True)}</code>
• {base_symbol}: <code>{format_number(analysis['base_liquidity'], decimals=2)}</code>
• {quote_symbol}: <code>{format_number(analysis['quote_liquidity'], decimals=2)}</code>

"""
        
        # Pool parameters
        report_message += f"""⚙️ <b>Pool Parameters</b>
• Fee: <code>{analysis['fee_pct']:.2f}%</code>
• Bin Step: <code>{analysis['bin_step']}</code>
• Active Bin: <code>{analysis['active_bin_id']}</code>
• Total Bins: <code>{analysis['num_bins']}</code>

"""
        
        # Liquidity distribution with smart price formatting
        min_price_str = format_price(analysis['price_range']['min'])
        max_price_str = format_price(analysis['price_range']['max'])
        report_message += f"""📈 <b>Liquidity Distribution</b>
• Price Range: <code>{min_price_str} - {max_price_str}</code>
"""
        
        # Show liquidity at different ranges instead of key levels
        if analysis.get('liquidity_distribution'):
            report_message += f"\n🎯 <b>Liquidity Around Price</b>\n"
            for pct in [5, 10, 15, 20]:
                if pct in analysis['liquidity_distribution']:
                    liquidity_pct = analysis['liquidity_distribution'][pct]
                    report_message += f"• ±{pct}%: <code>{liquidity_pct:.1f}%</code> of total liquidity\n"
        
        report_message += "\n"
    
    # Add footer
    report_message += f"""━━━━━━━━━━━━━━━━━━━━
💡 <i>Data from Gateway Meteora CLMM | QuantsLab</i>"""
    
    return report_message

def create_liquidity_chart(analysis: Dict) -> go.Figure:
    """Create liquidity distribution chart for a pool"""
    
    if 'bins_df' not in analysis or analysis['bins_df'].empty:
        return None
        
    bins_df = analysis['bins_df']
    
    # Create single chart figure
    fig = go.Figure()
    
    # Color bins based on active bin
    active_bin_id = analysis['active_bin_id']
    colors = ['red' if bid < active_bin_id else 'green' if bid > active_bin_id else 'yellow' 
              for bid in bins_df['binId']]
    
    # Main liquidity chart
    fig.add_trace(
        go.Bar(
            x=bins_df['price'],
            y=bins_df['total_value'],
            name='Liquidity (USD)',
            marker=dict(color=colors, opacity=0.7),
            hovertemplate=(
                'Price: $%{x:.8f}<br>'
                'Liquidity: $%{y:,.0f}<br>'
                'Bin ID: %{customdata}<br>'
                '<extra></extra>'
            ),
            customdata=bins_df['binId']
        )
    )
    
    # Add current price line
    fig.add_vline(
        x=analysis['current_price'],
        line_dash="dash",
        line_color="blue",
        line_width=2,
        annotation_text=f"Current Price: {format_price(analysis['current_price'])}",
        annotation_position="top"
    )
    
    # Update layout
    fig.update_xaxes(title_text="Price (USD)", showgrid=True, gridwidth=1, gridcolor='#E0E0E0')
    fig.update_yaxes(title_text="Liquidity (USD)")
    
    # Create comprehensive title with token pair, fee tier, and bin step
    base_token = analysis.get('base_token', 'BASE')
    quote_token = analysis.get('quote_token', 'QUOTE')
    fee_tier = f"{analysis['fee_pct']:.2f}%"
    bin_step = analysis['bin_step']
    timestamp = datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M UTC')
    
    fig.update_layout(
        title=dict(
            text=f"{base_token}/{quote_token} • {fee_tier} Fee • Bin Step: {bin_step}<br><sub>{timestamp}</sub>",
            x=0.5,
            xanchor='center',
            font=dict(size=18, family='Arial Bold')
        ),
        height=600,
        width=1200,
        showlegend=True,
        template="plotly_white",
        hovermode='x unified'
    )
    
    return fig

print(f"📊 Configuration loaded:")
print(f"  - Gateway URL: {GATEWAY_URL}")
print(f"  - Gateway Host: {gateway_config.gateway_api_host}")
print(f"  - Gateway Port: {gateway_config.gateway_api_port}")
print(f"  - SSL Enabled: {gateway_config.gateway_use_ssl}")
print(f"  - Network: {NETWORK}")
print(f"  - Update Interval: {UPDATE_INTERVAL_SECONDS}s")
print(f"  - Pool Addresses: {len(POOL_ADDRESSES)} configured")
for i, addr in enumerate(POOL_ADDRESSES, 1):
    print(f"    {i}. {addr}")
print(f"🔔 Enabled notifiers: {', '.join(enabled_notifiers) if enabled_notifiers else 'None configured'}")
print(f"✅ Gateway HTTP Client ready")
print("✅ Meteora Pool Analyzer initialized with Gateway HTTP Client")
print("📤 Telegram reports will be collected and sent at the end")
print("🗂️ Token cache initialized for efficient lookups")

📊 Configuration loaded:
  - Gateway URL: http://localhost:15888
  - Gateway Host: localhost
  - Gateway Port: 15888
  - SSL Enabled: False
  - Network: mainnet-beta
  - Update Interval: 300s
  - Pool Addresses: 3 configured
    1. 2sf5NYcY4zUPXUSmG6f66mskb24t5F8S11pC1Nz5nQT3
    2. C8Gr6AUuq9hEdSYJzoEpNcdjpojPZwqG5MtQbeouNNwg
    3. 9d9mb8kooFfaD3SctgZtkxQypkshx6ezhbKio89ixyy2
🔔 Enabled notifiers: telegram
✅ Gateway HTTP Client ready
✅ Meteora Pool Analyzer initialized with Gateway HTTP Client
📤 Telegram reports will be collected and sent at the end
🗂️ Token cache initialized for efficient lookups


In [188]:
# 📡 Fetch Pool Data and Token Metadata from Gateway
print(f"🔄 Fetching pool data from Gateway...")
print(f"📍 Gateway URL: {GATEWAY_URL}")
print(f"🔗 Network: {NETWORK}")
print(f"🏊 Pools to analyze: {len(POOL_ADDRESSES)}\n")

# Test gateway connection
try:
    is_online = await gateway_client.ping_gateway()
    if is_online:
        print("✅ Gateway connection successful\n")
    else:
        print("⚠️ Gateway is offline, trying to connect...\n")
except Exception as e:
    print(f"❌ Gateway connection error: {e}\n")
    is_online = False

# Fetch all pool data if online
all_pool_analyses = []
if is_online:
    print("🔄 Fetching data for all pools with token metadata...")
    
    for i, pool_address in enumerate(POOL_ADDRESSES, 1):
        print(f"\n📊 Pool {i}/{len(POOL_ADDRESSES)}: {pool_address}")
        
        try:
            # Fetch pool data
            pool_data = await pool_analyzer.fetch_pool_info(pool_address, NETWORK)
            
            if pool_data:
                print(f"✅ Raw pool data fetched")
                
                # Display raw pool info
                print(f"🏊 RAW POOL DATA:")
                print(f"  Pool Address: {pool_data['address']}")
                print(f"  Base Token Address: {pool_data['baseTokenAddress']}")
                print(f"  Quote Token Address: {pool_data['quoteTokenAddress']}")
                print(f"  Current Price: ${pool_data['price']:.4f}")
                print(f"  Fee: {pool_data['feePct']:.2f}%")
                print(f"  Bin Step: {pool_data['binStep']}")
                print(f"  Active Bin ID: {pool_data['activeBinId']}")
                print(f"  Total Bins: {len(pool_data.get('bins', []))}")
                
                # Fetch token metadata
                print(f"🔍 Looking up token metadata...")
                base_token_info = None
                quote_token_info = None
                base_symbol = pool_data['baseTokenAddress']  # Use address as fallback
                quote_symbol = pool_data['quoteTokenAddress']  # Use address as fallback
                
                # Lookup base token
                if pool_data.get('baseTokenAddress'):
                    try:
                        base_token_info = await pool_analyzer.fetch_token_info(pool_data['baseTokenAddress'], NETWORK)
                        if base_token_info and 'symbol' in base_token_info:
                            base_symbol = base_token_info['symbol']
                            print(f"  ✅ Base Token: {base_token_info['name']} ({base_token_info['symbol']})")
                            print(f"    Address: {base_token_info['address']}")
                            print(f"    Decimals: {base_token_info['decimals']}")
                        else:
                            print(f"  ⚠️ Base Token: Could not fetch metadata")
                    except Exception as e:
                        print(f"  ❌ Base Token lookup error: {e}")
                
                # Lookup quote token  
                if pool_data.get('quoteTokenAddress'):
                    try:
                        quote_token_info = await pool_analyzer.fetch_token_info(pool_data['quoteTokenAddress'], NETWORK)
                        if quote_token_info and 'symbol' in quote_token_info:
                            quote_symbol = quote_token_info['symbol']
                            print(f"  ✅ Quote Token: {quote_token_info['name']} ({quote_token_info['symbol']})")
                            print(f"    Address: {quote_token_info['address']}")
                            print(f"    Decimals: {quote_token_info['decimals']}")
                        else:
                            print(f"  ⚠️ Quote Token: Could not fetch metadata")
                    except Exception as e:
                        print(f"  ❌ Quote Token lookup error: {e}")
                
                # Create token pair name
                token_pair = f"{base_symbol}/{quote_symbol}"
                pool_name = f"{token_pair} Pool"
                
                print(f"🏷️ Identified Pool: {token_pair}")
                
                # Create metadata with token info
                metadata = {
                    'pool_name': pool_name,
                    'token_pair': token_pair,
                    'base_token_symbol': base_symbol,
                    'quote_token_symbol': quote_symbol,
                    'base_token_info': base_token_info,
                    'quote_token_info': quote_token_info
                }
                
                # Perform analysis
                analysis = pool_analyzer.analyze_pool(pool_data, metadata)
                all_pool_analyses.append(analysis)
                
                print(f"📊 Analysis Summary:")
                print(f"  Price: ${analysis['current_price']:.4f}")
                print(f"  Liquidity: {format_number(analysis['total_liquidity_usd'], is_currency=True)}")
                print(f"  Base ({base_symbol}): {format_number(analysis['base_liquidity'], decimals=2)}")
                print(f"  Quote ({quote_symbol}): {format_number(analysis['quote_liquidity'], decimals=2)}")
                print(f"  Bins: {analysis['num_bins']}")
                
            else:
                print(f"❌ Failed to fetch data for pool {pool_address}")
                
        except Exception as e:
            print(f"❌ Error analyzing pool {pool_address}: {e}")
    
    print(f"\n✅ Successfully analyzed {len(all_pool_analyses)} out of {len(POOL_ADDRESSES)} pools")
    
    if all_pool_analyses:
        total_liquidity = sum(a['total_liquidity_usd'] for a in all_pool_analyses)
        print(f"💰 Total Liquidity Across All Pools: {format_number(total_liquidity, is_currency=True)}")
        
        # Display summary with token pairs
        print(f"\n📊 POOL ANALYSIS SUMMARY:")
        print("=" * 80)
        for analysis in all_pool_analyses:
            token_pair = analysis.get('token_pair', analysis['pool_name'])
            print(f"{token_pair:<20} | "
                  f"${analysis['current_price']:<8.4f} | "
                  f"{format_number(analysis['total_liquidity_usd'], is_currency=True):<12} | "
                  f"{analysis['num_bins']:>4} bins")
        print("=" * 80)
        
else:
    print("❌ Cannot fetch pool data - Gateway is offline")
    all_pool_analyses = []

🔄 Fetching pool data from Gateway...
📍 Gateway URL: http://localhost:15888
🔗 Network: mainnet-beta
🏊 Pools to analyze: 3

✅ Gateway connection successful

🔄 Fetching data for all pools with token metadata...

📊 Pool 1/3: 2sf5NYcY4zUPXUSmG6f66mskb24t5F8S11pC1Nz5nQT3
✅ Raw pool data fetched
🏊 RAW POOL DATA:
  Pool Address: 2sf5NYcY4zUPXUSmG6f66mskb24t5F8S11pC1Nz5nQT3
  Base Token Address: So11111111111111111111111111111111111111112
  Quote Token Address: EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v
  Current Price: $211.7706
  Fee: 0.04%
  Bin Step: 100
  Active Bin ID: -156
  Total Bins: 141
🔍 Looking up token metadata...
  ✅ Base Token: Wrapped Solana (SOL)
    Address: So11111111111111111111111111111111111111112
    Decimals: 9
  ✅ Quote Token: USDC (USDC)
    Address: EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v
    Decimals: 6
🏷️ Identified Pool: SOL/USDC
📊 Analysis Summary:
  Price: $211.7706
  Liquidity: $64.43K
  Base (SOL): 151.25
  Quote (USDC): 32.40K
  Bins: 141

📊 Pool 2/3: 

2025-09-23 20:28:40,274 - hummingbot.core.gateway.gateway_http_client - WARNING - Call to http://localhost:15888/tokens/6p6xgHyF7AeE6TZkSmFsko444wqoP15icUSqi2jfGiPN failed. See logs for more details.


✅ Raw pool data fetched
🏊 RAW POOL DATA:
  Pool Address: 9d9mb8kooFfaD3SctgZtkxQypkshx6ezhbKio89ixyy2
  Base Token Address: 6p6xgHyF7AeE6TZkSmFsko444wqoP15icUSqi2jfGiPN
  Quote Token Address: EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v
  Current Price: $7.6135
  Fee: 0.10%
  Bin Step: 50
  Active Bin ID: 407
  Total Bins: 141
🔍 Looking up token metadata...
❌ Error fetching token info for 6p6xgHyF7AeE6TZkSmFsko444wqoP15icUSqi2jfGiPN: Error on GET http://localhost:15888/tokens/6p6xgHyF7AeE6TZkSmFsko444wqoP15icUSqi2jfGiPN Error: NotFoundError
  ⚠️ Base Token: Could not fetch metadata
  ✅ Quote Token: USDC (USDC)
    Address: EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v
    Decimals: 6
🏷️ Identified Pool: 6p6xgHyF7AeE6TZkSmFsko444wqoP15icUSqi2jfGiPN/USDC
📊 Analysis Summary:
  Price: $7.6135
  Liquidity: $317.36M
  Base (6p6xgHyF7AeE6TZkSmFsko444wqoP15icUSqi2jfGiPN): 15.46M
  Quote (USDC): 199.64M
  Bins: 141

✅ Successfully analyzed 3 out of 3 pools
💰 Total Liquidity Across All Pools: $31

In [189]:
# 🚀 Generate Pool Analysis Report with Token Lookup (Preview Only)
if all_pool_analyses:
    print("🔍 Looking up token symbols and generating report...")
    
    # Generate report with token symbol lookup
    report_message = await generate_pool_report(all_pool_analyses)
    
    # Store the report for sending later
    telegram_reports['report_message'] = report_message
    
    print("📤 Pool analysis report generated successfully")
    print(f"📊 Report includes {len(all_pool_analyses)} pool(s) with token symbols")
    print("📋 Report stored for Telegram sending at the end\n")
    
    # Display detailed analysis for each pool
    print("📊 DETAILED POOL ANALYSIS")
    print("=" * 80)
    for i, analysis in enumerate(all_pool_analyses, 1):
        print(f"\n📊 POOL {i} - {analysis['pool_name']}")
        print("=" * 60)
        print(f"Address: {analysis['address']}")
        print(f"Current Price: {format_price(analysis['current_price'])}")
        print(f"Total Liquidity (USD): {format_number(analysis['total_liquidity_usd'], is_currency=True)}")
        print(f"Base Liquidity: {format_number(analysis['base_liquidity'], decimals=2)} {analysis['base_token']}")
        print(f"Quote Liquidity: {format_number(analysis['quote_liquidity'], decimals=2)} {analysis['quote_token']}")
        print(f"Liquidity Concentration: {analysis['liquidity_concentration']:.1f}% (within ±5 bins)")
        print(f"Number of Bins: {analysis['num_bins']}")
        print(f"Price Range: {format_price(analysis['price_range']['min'])} - {format_price(analysis['price_range']['max'])}")
        print(f"Fee: {analysis['fee_pct']:.2f}%")
        print(f"Bin Step: {analysis['bin_step']}")
        print(f"Active Bin ID: {analysis['active_bin_id']}")
        
        if analysis.get('key_levels'):
            print(f"\n🎯 KEY LIQUIDITY LEVELS:")
            for j, level in enumerate(analysis['key_levels'], 1):
                print(f"  {j}. Price: {format_price(level['price'])} | Liquidity: {format_number(level['total_value'], is_currency=True)}")
        
        print("=" * 60)
    
    print("\n📋 TELEGRAM REPORT PREVIEW:")
    print("=" * 50)
    print(report_message.replace('<b>', '**').replace('</b>', '**')
          .replace('<code>', '`').replace('</code>', '`')
          .replace('<i>', '_').replace('</i>', '_'))
    print("=" * 50)
        
else:
    report_message = await generate_pool_report([])
    print("⚠️ No pool data available for report generation")

🔍 Looking up token symbols and generating report...
📤 Pool analysis report generated successfully
📊 Report includes 3 pool(s) with token symbols
📋 Report stored for Telegram sending at the end

📊 DETAILED POOL ANALYSIS

📊 POOL 1 - SOL/USDC Pool
Address: 2sf5NYcY4zUPXUSmG6f66mskb24t5F8S11pC1Nz5nQT3
Current Price: $211.77
Total Liquidity (USD): $64.43K
Base Liquidity: 151.25 SOL
Quote Liquidity: 32.40K USDC
Liquidity Concentration: 9.9% (within ±5 bins)
Number of Bins: 141
Price Range: $106.58 - $429.22
Fee: 0.04%
Bin Step: 100
Active Bin ID: -156

🎯 KEY LIQUIDITY LEVELS:
  1. Price: $224.80 | Liquidity: $815.66
  2. Price: $227.05 | Liquidity: $812.26
  3. Price: $229.32 | Liquidity: $782.04
  4. Price: $231.61 | Liquidity: $778.85
  5. Price: $233.93 | Liquidity: $774.44

📊 POOL 2 - JUP/SOL Pool
Address: C8Gr6AUuq9hEdSYJzoEpNcdjpojPZwqG5MtQbeouNNwg
Current Price: $0.0021834
Total Liquidity (USD): $14.91K
Base Liquidity: 3.93M JUP
Quote Liquidity: 6.33K SOL
Liquidity Concentration: 20.2

In [190]:
# 🎨 Generate Liquidity Distribution Charts with Token Symbols (Save for Later)
if all_pool_analyses:
    print("📊 Generating liquidity distribution charts with token symbols...\n")
    
    # Clear any existing chart files from previous runs
    telegram_reports['chart_files'] = []
    
    for i, analysis in enumerate(all_pool_analyses, 1):
        print(f"🎨 Chart {i}/{len(all_pool_analyses)}: {analysis['pool_name']}")
        
        # Use cached token symbols from initial pool fetching
        base_symbol = analysis['base_token']
        quote_symbol = analysis['quote_token']
        
        # Use cache if available and token symbols are still addresses
        if analysis.get('base_token_address') and len(base_symbol) > 10:
            if analysis['base_token_address'] in token_cache:
                cached_info = token_cache[analysis['base_token_address']]
                if cached_info and 'symbol' in cached_info:
                    base_symbol = cached_info['symbol']
                    
        if analysis.get('quote_token_address') and len(quote_symbol) > 10:
            if analysis['quote_token_address'] in token_cache:
                cached_info = token_cache[analysis['quote_token_address']]
                if cached_info and 'symbol' in cached_info:
                    quote_symbol = cached_info['symbol']
        
        # Update analysis with proper symbols for chart generation
        analysis_with_symbols = analysis.copy()
        analysis_with_symbols['base_token'] = base_symbol
        analysis_with_symbols['quote_token'] = quote_symbol
        analysis_with_symbols['token_pair'] = f"{base_symbol}/{quote_symbol}"
        
        # Create chart for this pool with proper token symbols
        fig = create_liquidity_chart(analysis_with_symbols)
        
        if fig:
            print(f"  ✅ Chart generated for {analysis_with_symbols['token_pair']}")
            fig.show()
            
            # Save chart as PNG with token pair, fee tier, and bin step in filename
            safe_pair_name = analysis_with_symbols['token_pair'].replace('/', '_')
            fee_tier = f"{analysis['fee_pct']:.2f}".replace('.', 'p')  # 0.04 -> 0p04
            bin_step = analysis['bin_step']
            chart_filename = f"/tmp/meteora_{safe_pair_name}_{fee_tier}pct_bin{bin_step}_chart.png"
            
            try:
                fig.write_image(chart_filename, width=1200, height=800, scale=2)
                print(f"  💾 Chart saved: {chart_filename}")
                
                # Store chart info for Telegram sending later with enhanced caption
                chart_info = {
                    'filename': chart_filename,
                    'pool_name': analysis['pool_name'],
                    'token_pair': analysis_with_symbols['token_pair'],
                    'caption': f"""📊 <b>Liquidity Distribution Chart</b>
🏊 Pool: {analysis_with_symbols['token_pair']} • {analysis['fee_pct']:.2f}% Fee • Bin:{analysis['bin_step']}
💰 Total Liquidity: {format_number(analysis['total_liquidity_usd'], is_currency=True)}
📍 Current Price: ${analysis['current_price']:.4f}
🎯 Concentration: {analysis['liquidity_concentration']:.1f}% near price
⚙️ Active Bin: {analysis['active_bin_id']} | Total Bins: {analysis['num_bins']}

<i>Generated by QuantsLab Pool Analyzer</i>"""
                }
                telegram_reports['chart_files'].append(chart_info)
                print(f"  📋 Chart stored for Telegram sending")
                        
            except ImportError as e:
                print(f"  ⚠️ Kaleido not installed: {e}")
                print("  💡 Install with: pip install kaleido")
                
                # Save as HTML fallback
                html_filename = chart_filename.replace('.png', '.html')
                fig.write_html(html_filename)
                print(f"  📄 Saved as HTML: {html_filename}")
                
            except Exception as e:
                print(f"  ❌ Error saving chart: {e}")
        else:
            print(f"  ⚠️ No bin data available for {analysis['pool_name']}")
            
        print()  # Empty line between pools
        
    print(f"✅ Chart generation completed for {len(all_pool_analyses)} pool(s)")
    print(f"📋 {len(telegram_reports['chart_files'])} charts stored for Telegram sending")
else:
    print("⚠️ No pool analyses available for chart generation")

📊 Generating liquidity distribution charts with token symbols...

🎨 Chart 1/3: SOL/USDC Pool
  ✅ Chart generated for SOL/USDC


  💾 Chart saved: /tmp/meteora_SOL_USDC_0p04pct_bin100_chart.png
  📋 Chart stored for Telegram sending

🎨 Chart 2/3: JUP/SOL Pool
  ✅ Chart generated for JUP/SOL


  💾 Chart saved: /tmp/meteora_JUP_SOL_0p15pct_bin80_chart.png
  📋 Chart stored for Telegram sending

🎨 Chart 3/3: 6p6xgHyF7AeE6TZkSmFsko444wqoP15icUSqi2jfGiPN/USDC Pool
  ✅ Chart generated for 6p6xgHyF7AeE6TZkSmFsko444wqoP15icUSqi2jfGiPN/USDC


  💾 Chart saved: /tmp/meteora_6p6xgHyF7AeE6TZkSmFsko444wqoP15icUSqi2jfGiPN_USDC_0p10pct_bin50_chart.png
  📋 Chart stored for Telegram sending

✅ Chart generation completed for 3 pool(s)
📋 3 charts stored for Telegram sending


In [194]:
# 📊 Generate CSV Reports with Token Symbols (Save for Later)
if all_pool_analyses:
    print("📋 Generating detailed CSV reports with token symbols...\n")
    
    # Clear any existing CSV files from previous runs
    telegram_reports['csv_files'] = []
    
    for i, analysis in enumerate(all_pool_analyses, 1):
        if 'bins_df' in analysis and not analysis['bins_df'].empty:
            print(f"📄 CSV Report {i}/{len(all_pool_analyses)}: {analysis['pool_name']}")
            
            # Use cached token symbols from initial pool fetching
            base_symbol = analysis['base_token']
            quote_symbol = analysis['quote_token']
            
            # Use cache if available and token symbols are still addresses
            if analysis.get('base_token_address') and len(base_symbol) > 10:
                if analysis['base_token_address'] in token_cache:
                    cached_info = token_cache[analysis['base_token_address']]
                    if cached_info and 'symbol' in cached_info:
                        base_symbol = cached_info['symbol']
                        
            if analysis.get('quote_token_address') and len(quote_symbol) > 10:
                if analysis['quote_token_address'] in token_cache:
                    cached_info = token_cache[analysis['quote_token_address']]
                    if cached_info and 'symbol' in cached_info:
                        quote_symbol = cached_info['symbol']
            
            token_pair = f"{base_symbol}/{quote_symbol}"
            
            # Prepare detailed DataFrame
            detailed_df = analysis['bins_df'].copy()
            
            # Add calculated fields with proper token symbols, fee tier, and bin step
            detailed_df['token_pair'] = token_pair
            detailed_df['base_token_symbol'] = base_symbol
            detailed_df['quote_token_symbol'] = quote_symbol
            detailed_df['fee_pct'] = analysis['fee_pct']
            detailed_df['bin_step'] = analysis['bin_step']
            detailed_df['pool_name'] = analysis['pool_name']
            detailed_df['pool_address'] = analysis['address']
            detailed_df['base_token_address'] = analysis.get('base_token_address', '')
            detailed_df['quote_token_address'] = analysis.get('quote_token_address', '')
            detailed_df['current_price'] = analysis['current_price']
            detailed_df['is_active_bin'] = detailed_df['binId'] == analysis['active_bin_id']
            detailed_df['distance_from_price'] = ((detailed_df['price'] - analysis['current_price']) / analysis['current_price'] * 100)
            detailed_df['total_value_usd'] = (
                detailed_df['baseTokenAmount'] * analysis['current_price'] + 
                detailed_df['quoteTokenAmount']
            )
            detailed_df['liquidity_share_pct'] = (detailed_df['total_value_usd'] / detailed_df['total_value_usd'].sum() * 100)
            detailed_df['report_timestamp'] = datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M:%S UTC')
            
            # Reorder columns with token information, fee tier, and bin step first
            column_order = [
                'token_pair', 'base_token_symbol', 'quote_token_symbol', 'fee_pct', 'bin_step',
                'pool_name', 'pool_address', 'base_token_address', 'quote_token_address',
                'binId', 'price', 'is_active_bin',
                'baseTokenAmount', 'quoteTokenAmount', 'total_value_usd',
                'liquidity_share_pct', 'distance_from_price', 'current_price',
                'report_timestamp'
            ]
            
            detailed_df = detailed_df[column_order]
            
            # Save to CSV with token pair, fee tier, and bin step in filename
            timestamp_str = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M")
            safe_pair_name = token_pair.replace('/', '_')
            fee_tier = f"{analysis['fee_pct']:.2f}".replace('.', 'p')  # 0.04 -> 0p04
            bin_step = analysis['bin_step']
            csv_filename = f"/tmp/meteora_{safe_pair_name}_{fee_tier}pct_bin{bin_step}_{timestamp_str}.csv"
            detailed_df.to_csv(csv_filename, index=False, float_format='%.8f')
            
            print(f"  ✅ CSV saved: {csv_filename}")
            print(f"  📊 Records: {len(detailed_df)}")
            
            # Display sample data with token symbols, fee tier, and bin step
            print(f"  📋 CSV Preview (top 3 rows):")
            display_df = detailed_df.head(3)[['token_pair', 'fee_pct', 'bin_step', 'binId', 'price', 'total_value_usd', 'liquidity_share_pct']]
            for _, row in display_df.iterrows():
                print(f"    {row['token_pair']} {row['fee_pct']:.2f}% Bin{row['bin_step']} | Bin {row['binId']}: ${row['price']:.2f} | {format_number(row['total_value_usd'], is_currency=True)} ({row['liquidity_share_pct']:.2f}%)")
            
            # Store CSV info for Telegram sending later with enhanced caption
            csv_info = {
                'filename': csv_filename,
                'pool_name': analysis['pool_name'],
                'token_pair': token_pair,
                'caption': f"""📋 <b>Detailed Pool Data Export</b>
🏊 Pool: {token_pair} • {analysis['fee_pct']:.2f}% Fee • Bin:{analysis['bin_step']}
📅 {datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M UTC')}

📊 Dataset Summary:
• Total Bins: {len(detailed_df)}
• Active Bin: {analysis['active_bin_id']}
• Price Range: ${detailed_df['price'].min():.2f} - ${detailed_df['price'].max():.2f}
• Total Liquidity: {format_number(analysis['total_liquidity_usd'], is_currency=True)}
• Fee Tier: {analysis['fee_pct']:.2f}% | Bin Step: {analysis['bin_step']}

🔍 <i>Complete bin-level data for {base_symbol}/{quote_symbol} analysis</i>"""
            }
            telegram_reports['csv_files'].append(csv_info)
            print(f"  📋 CSV stored for Telegram sending")
            
            print()  # Empty line between pools
            
        else:
            print(f"  ⚠️ No bin data available for {analysis['pool_name']}")
    
    print(f"✅ CSV report generation completed for all pools")
    print(f"📋 {len(telegram_reports['csv_files'])} CSV files stored for Telegram sending")
else:
    print("⚠️ No pool analyses available for CSV export")

📋 Generating detailed CSV reports with token symbols...

📄 CSV Report 1/3: SOL/USDC Pool
  ✅ CSV saved: /tmp/meteora_SOL_USDC_0p04pct_bin100_20250924_0329.csv
  📊 Records: 141
  📋 CSV Preview (top 3 rows):
    SOL/USDC 0.04% Bin100 | Bin -225: $106.58 | $158.94 (0.27%)
    SOL/USDC 0.04% Bin100 | Bin -224: $107.65 | $163.19 (0.28%)
    SOL/USDC 0.04% Bin100 | Bin -223: $108.73 | $180.27 (0.31%)
  📋 CSV stored for Telegram sending

📄 CSV Report 2/3: JUP/SOL Pool
  ✅ CSV saved: /tmp/meteora_JUP_SOL_0p15pct_bin80_20250924_0329.csv
  📊 Records: 141
  📋 CSV Preview (top 3 rows):
    JUP/SOL 0.15% Bin80 | Bin 28: $0.00 | $0.00 (0.00%)
    JUP/SOL 0.15% Bin80 | Bin 29: $0.00 | $0.00 (0.00%)
    JUP/SOL 0.15% Bin80 | Bin 30: $0.00 | $0.06 (0.00%)
  📋 CSV stored for Telegram sending

📄 CSV Report 3/3: 6p6xgHyF7AeE6TZkSmFsko444wqoP15icUSqi2jfGiPN/USDC Pool
  ✅ CSV saved: /tmp/meteora_6p6xgHyF7AeE6TZkSmFsko444wqoP15icUSqi2jfGiPN_USDC_0p10pct_bin50_20250924_0329.csv
  📊 Records: 141
  📋 CSV Previe

In [195]:
# 📤 Send All Reports to Telegram
print("🚀 SENDING ALL REPORTS TO TELEGRAM")
print("=" * 60)

telegram_notifier = notification_manager.get_notifier('telegram')
if not telegram_notifier:
    print("❌ Telegram notifier not configured")
    print("💡 Configure Telegram in .env to send reports")
else:
    # Summary of what will be sent
    text_count = 1 if telegram_reports['report_message'] else 0
    chart_count = len(telegram_reports['chart_files'])
    csv_count = len(telegram_reports['csv_files'])
    total_items = text_count + chart_count + csv_count
    
    print(f"📊 Sending {total_items} items to Telegram:")
    print(f"  • {text_count} text report")
    print(f"  • {chart_count} chart files")
    print(f"  • {csv_count} CSV files")
    print()
    
    sent_count = 0
    failed_count = 0
    
    # Send text report first
    if telegram_reports['report_message']:
        print("1️⃣ Sending text report...")
        try:
            notification_msg = NotificationMessage(
                title="Meteora Pool Analysis",
                message=telegram_reports['report_message'],
                level="info"
            )
            
            if await telegram_notifier.send_notification(notification_msg):
                print("  ✅ Text report sent successfully")
                sent_count += 1
            else:
                print("  ❌ Failed to send text report")
                failed_count += 1
        except Exception as e:
            print(f"  ❌ Error sending text report: {e}")
            failed_count += 1
        print()
    
    # Send chart files
    if telegram_reports['chart_files']:
        print("2️⃣ Sending chart files...")
        for i, chart_info in enumerate(telegram_reports['chart_files'], 1):
            print(f"  📊 Chart {i}/{chart_count}: {chart_info['pool_name']}")
            try:
                if await telegram_notifier.send_photo(chart_info['filename'], chart_info['caption']):
                    print(f"    ✅ Chart sent successfully")
                    sent_count += 1
                else:
                    print(f"    ❌ Failed to send chart")
                    failed_count += 1
            except Exception as e:
                print(f"    ❌ Error sending chart: {e}")
                failed_count += 1
        print()
    
    # Send CSV files
    if telegram_reports['csv_files']:
        print("3️⃣ Sending CSV files...")
        for i, csv_info in enumerate(telegram_reports['csv_files'], 1):
            print(f"  📄 CSV {i}/{csv_count}: {csv_info['pool_name']}")
            try:
                if await telegram_notifier.send_document(csv_info['filename'], csv_info['caption']):
                    print(f"    ✅ CSV sent successfully")
                    sent_count += 1
                else:
                    print(f"    ❌ Failed to send CSV")
                    failed_count += 1
            except Exception as e:
                print(f"    ❌ Error sending CSV: {e}")
                failed_count += 1
        print()
    
    # Final summary
    print("📊 TELEGRAM SENDING SUMMARY")
    print("=" * 40)
    print(f"✅ Successfully sent: {sent_count}/{total_items}")
    print(f"❌ Failed to send: {failed_count}/{total_items}")
    
    if sent_count > 0:
        print(f"\n🎉 Check your Telegram for {sent_count} new messages!")
    
    if failed_count > 0:
        print(f"\n⚠️ {failed_count} items failed to send - check error messages above")

🚀 SENDING ALL REPORTS TO TELEGRAM
📊 Sending 7 items to Telegram:
  • 1 text report
  • 3 chart files
  • 3 CSV files

1️⃣ Sending text report...
  ✅ Text report sent successfully

2️⃣ Sending chart files...
  📊 Chart 1/3: SOL/USDC Pool
    ✅ Chart sent successfully
  📊 Chart 2/3: JUP/SOL Pool
    ✅ Chart sent successfully
  📊 Chart 3/3: 6p6xgHyF7AeE6TZkSmFsko444wqoP15icUSqi2jfGiPN/USDC Pool
    ✅ Chart sent successfully

3️⃣ Sending CSV files...
  📄 CSV 1/3: SOL/USDC Pool
    ✅ CSV sent successfully
  📄 CSV 2/3: JUP/SOL Pool
    ✅ CSV sent successfully
  📄 CSV 3/3: 6p6xgHyF7AeE6TZkSmFsko444wqoP15icUSqi2jfGiPN/USDC Pool
    ✅ CSV sent successfully

📊 TELEGRAM SENDING SUMMARY
✅ Successfully sent: 7/7
❌ Failed to send: 0/7

🎉 Check your Telegram for 7 new messages!


In [197]:
# 🔒 Cleanup and Execution Summary
print("✅ Meteora pool analysis completed successfully\n")

# Clean up temporary files (optional)
import os
import glob

temp_files = glob.glob('/tmp/meteora_*.png') + glob.glob('/tmp/meteora_*.html') + glob.glob('/tmp/meteora_*.csv')
cleanup_count = 0

for file_path in temp_files:
    try:
        # Keep the most recent files
        file_age = datetime.now() - datetime.fromtimestamp(os.path.getctime(file_path))
        if file_age.days > 0:  # Clean files older than 1 day
            os.remove(file_path)
            cleanup_count += 1
            print(f"🗑️ Cleaned up: {os.path.basename(file_path)}")
    except Exception as e:
        print(f"⚠️ Could not remove {file_path}: {e}")

if cleanup_count > 0:
    print(f"\n🧹 Cleaned up {cleanup_count} old temporary files")

# Execution Summary
print("\n📊 EXECUTION SUMMARY")
print("=" * 50)
print(f"🌐 Data Source: Gateway API ({GATEWAY_URL})")
print(f"📡 Using: Hummingbot Gateway HTTP Client")
print(f"🔗 Network: Solana {NETWORK}")

if 'pool_analysis' in locals() and pool_analysis:
    print(f"🏊 Pool Analyzed: {pool_analysis['pool_name']}")
    print(f"💰 Total Liquidity: {format_number(pool_analysis['total_liquidity_usd'], is_currency=True)}")
    print(f"📤 Reports Sent: {'✅ Yes' if enabled_notifiers else '❌ No notifiers configured'}")
    print(f"📊 Charts Generated: ✅ Yes")
    print(f"📋 CSV Exports: ✅ Yes")
elif 'all_pool_analyses' in locals() and all_pool_analyses:
    total_liquidity = sum(a['total_liquidity_usd'] for a in all_pool_analyses)
    print(f"🏊 Pools Analyzed: {len(all_pool_analyses)}")
    print(f"💰 Total Liquidity: {format_number(total_liquidity, is_currency=True)}")
    print(f"📤 Reports Sent: {'✅ Yes' if enabled_notifiers else '❌ No notifiers configured'}")
else:
    print(f"⚠️ No pools analyzed - check Gateway connection")

print("\n🎉 Meteora pool analysis complete!")

if 'telegram' in enabled_notifiers:
    print("📱 Check your Telegram for delivered reports, charts, and data files")
else:
    print("💡 Configure Telegram in .env to receive automated reports")

✅ Meteora pool analysis completed successfully


📊 EXECUTION SUMMARY
🌐 Data Source: Gateway API (http://localhost:15888)
📡 Using: Hummingbot Gateway HTTP Client
🔗 Network: Solana mainnet-beta
🏊 Pool Analyzed: SOL-USDC
💰 Total Liquidity: $64.74K
📤 Reports Sent: ✅ Yes
📊 Charts Generated: ✅ Yes
📋 CSV Exports: ✅ Yes

🎉 Meteora pool analysis complete!
📱 Check your Telegram for delivered reports, charts, and data files
